##### Name: Mahmoud Hamdy Mahmoud Hashim 
#### BN: 24 in Sec: 2

In [195]:
import numpy as np
import pandas as pd
import sklearn

In [196]:
data = pd.read_csv("cardio_train_ML.csv", sep=";") #Reading data and specify the separator which is a semicolon ";"

## Preprocessing

In [197]:
data['age'] //= 365
data = data.drop(['id'], axis=1)

In [198]:
#Inserting new feature replacable to height and weight and representaple to obesity
data.insert(3, 'BMI',data['weight']/(data['height']/100)**2)

In [199]:
data

,age,gender,height,BMI,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,50,2,168,21.967120,62.0,110,80,1,1,0,0,1,0
1,55,1,156,34.927679,85.0,140,90,3,1,0,0,1,1
2,51,1,165,23.507805,64.0,130,70,3,1,0,0,0,1
3,48,2,169,28.710479,82.0,150,100,1,1,0,0,1,1
4,47,1,156,23.011177,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,52,2,168,26.927438,76.0,120,80,1,1,1,0,1,0
69996,61,1,158,50.472681,126.0,140,90,2,2,0,0,1,1
69997,52,2,183,31.353579,105.0,180,90,3,1,0,1,0,1
69998,61,1,163,27.099251,72.0,135,80,1,2,0,0,0,1


In [200]:
data['ap_hi'] = abs(data['ap_hi'])
data['ap_lo'] = abs(data['ap_lo'])
print(data['ap_hi'].describe())
print(data['ap_lo'].describe())

count    70000.000000
mean       128.841429
std        153.991223
min          1.000000
25%        120.000000
50%        120.000000
75%        140.000000
max      16020.000000
Name: ap_hi, dtype: float64
count    70000.000000
mean        96.632414
std        188.471505
min          0.000000
25%         80.000000
50%         80.000000
75%         90.000000
max      11000.000000
Name: ap_lo, dtype: float64


In [201]:
#Convert it into classes to ease classification
data['BMI'] = pd.cut(data['BMI'], bins = [0, 18, 25, 30, 100], labels = [0,1,2,3])
data['age'] = pd.cut(data['age'], bins = [0, 20, 40, 60, 80], labels = [0,1,2,3])
data['ap_hi'] = pd.cut(data['ap_hi'], bins = [0, 100, 150, 180, 400], labels = [0,1,2,3])
data['ap_lo'] = pd.cut(data['ap_lo'], bins = [0, 60, 80, 100, 200], labels = [0,1,2,3])

In [202]:
data = data.drop(['weight', 'height'], axis = 1)

In [203]:
data["BMI"].value_counts()

1    26029
2    25073
3    18444
0      425
Name: BMI, dtype: int64

In [204]:
data

,age,gender,BMI,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,2,2,1,1,1,1,1,0,0,1,0
1,2,1,3,1,2,3,1,0,0,1,1
2,2,1,1,1,1,3,1,0,0,0,1
3,2,2,2,1,2,1,1,0,0,1,1
4,2,1,1,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
69995,2,2,2,1,1,1,1,1,0,1,0
69996,3,1,3,1,2,2,2,0,0,1,1
69997,2,2,3,2,2,3,1,0,1,0,1
69998,3,1,2,1,1,1,2,0,0,0,1


## Decision tree implementation

In [205]:
class desicion_tree:

    def entropy(self, target_column):  
        entropy = 0
        #We get the possible values and their counts for the feature in the data
        values, counts = np.unique(target_column, return_counts = True)
        #We sum the entropy for each class in the target_feature
        for i in range(len(values)):
            entropy += (counts[i])/(len(target_column)) * np.log2((counts[i])/(len(target_column)))
#         entropy=np.sum([-(counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(values))])
        return entropy

    def IG(self, data, feature, target):
        total_entropy = self.entropy(target)#Calculate the total entropy based on the target data (Call)
        #We get the possible values and their counts for the required feature in the data
        values, counts = np.unique(data[feature], return_counts = True)
        #suum all entropies for feature classes
        entropy_sum =np.sum([(counts[i]/np.sum(counts)) * self.entropy(data.where(data[feature]==values[i]).dropna()[target]) for i in range(len(values))])
        #Get the information gain
        return total_entropy - entropy_sum
    
    def train_id3(self,traindata,dataset,features,target="cardio",parent_node=None):
        #We check whether the feature is pure or not and return it's value when pure
        if len(np.unique(traindata[target])) == 1:
            return np.unique(traindata[target])[0]
        #When the train data is empty we return the most frequent label
        elif len(traindata)==0: 
            return np.unique(dataset[target])[np.argmax(np.unique(dataset[target],return_counts=True)[1])]    
        #Stopping criteria (stop growing the tree when no features available)
        if len(features)==0:
            return parent_node
        #Set the parent node to the most frequent feature
        parent_node=np.unique(traindata[target])[np.argmax(np.unique(traindata[target],return_counts=True)[1])]
        #Select the feature which best splits the dataset max IG
        info_gain=[self.IG(traindata,feature,target)for feature in features]
        maxGainidx=np.argmax(info_gain)
        maxGain=features[maxGainidx]
        #Initialize the tree structure with the max_gain feature
        tree={maxGain:{}}
        #Remove the max feature 
        features=[i for i in features if i!=maxGain]
        #Build branches for the classes of the chosen feature
        for val in np.unique(traindata[maxGain]):
            value = val
            newdata=traindata.where(traindata[maxGain]==value).dropna()
            #Grow the other branches of the tree
            subtree=self.train_id3(newdata,dataset,features,target,parent_node)
            tree[maxGain][value]=subtree
        return tree
    
    def predict(self, samples,tree,default = 1):  
        # Loop through each key in the query dictionary
        for key in list(samples.keys()):  
            # Check if the key is in the tree dictionary
            if key in list(tree.keys()):  
                try:  
                    # Try to get the result from the tree dictionary
                    result = tree[key][samples[key]]   
                except:  
                    # If there's an error, return the default value
                    return default  
                result = tree[key][samples[key]]  
                if isinstance(result,dict):  
                    return self.predict(samples,result)  
                else:  
                    return result 
    def accuracy_test(self, data,tree):  
        #Create new query instances by simply removing the target feature column from the original dataset and   
        #convert it to a dictionary by The to_dict() method which converts this subset of the DataFrame to 
        #a dictionary with each row represented as a dictionary with keys as column names and values as row values.
        queries = data.iloc[:,:-1].to_dict(orient = "records")  
        
        #Create an empty DataFrame in whose columns the prediction of the tree are stored  
        predicted = pd.DataFrame(columns=["predicted"])   
        
        #Calculate the prediction accuracy  
        for i in range(len(data)):  
            predicted.loc[i,"predicted"] = self.predict(queries[i],tree)
        print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data["cardio"])/len(data))*100,'%')


    def traintestsplit(self, dataset,size):
        size=int(size*dataset.shape[0])
        #Droping index to ensure that the indices ranges from zero
        train=dataset.iloc[:size].reset_index(drop=True)
        test=dataset.iloc[size:].reset_index(drop=True)
        return train,test


In [206]:
dt = desicion_tree()
train, test = dt.traintestsplit(data, 0.8)

In [207]:
keys = train.keys()

In [208]:
import time 
start = time.time()
tree = dt.train_id3(train, train, keys[:-1], keys[-1])
end = time.time()
end - start

229.13725423812866

In [209]:
dt.accuracy_test(test, tree)

The prediction accuracy is:  65.65714285714286 %


In [210]:
tree

{'gender': {1: {'alco': {0.0: {'smoke': {0.0: {'active': {0.0: {'gluc': {1.0: {'BMI': {0: {'cholesterol': {1.0: {'ap_lo': {0: {'ap_hi': {0: {'age': {1: 0.0,
                    2: 1.0,
                    3: 0.0}}}},
                1: {'ap_hi': {0: {'age': {2: 0.0}},
                  1: {'age': {1: 0.0, 2: 1.0}}}},
                2: {'age': {2: {'ap_hi': {1: 1.0}}}}}}}},
            1: {'cholesterol': {1.0: {'ap_hi': {0: {'age': {1: {'ap_lo': {0: 0.0,
                    1: 0.0,
                    2: 1.0}},
                  2: {'ap_lo': {0: 0.0, 1: 0.0}},
                  3: {'ap_lo': {1: 0.0, 2: 1.0}}}},
                1: {'age': {1: {'ap_lo': {0: 0.0, 1: 0.0, 2: 0.0}},
                  2: {'ap_lo': {0: 0.0, 1: 0.0, 2: 0.0}},
                  3: {'ap_lo': {0: 1.0, 1: 1.0, 2: 1.0}}}},
                2: {'age': {1: 1.0,
                  2: {'ap_lo': {0: 0.0, 1: 1.0, 2: 1.0, 3: 1.0}},
                  3: {'ap_lo': {1: 1.0, 2: 1.0}}}},
                3: 0.0}},
              2

In [211]:
train

,age,gender,BMI,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,2,2,1,1,1,1,1,0,0,1,0
1,2,1,3,1,2,3,1,0,0,1,1
2,2,1,1,1,1,3,1,0,0,0,1
3,2,2,2,1,2,1,1,0,0,1,1
4,2,1,1,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
55995,2,1,2,1,1,1,1,0,0,1,1
55996,2,2,2,1,2,1,3,1,0,0,1
55997,2,1,1,0,0,1,1,0,0,1,0
55998,2,1,1,0,0,2,1,0,0,1,1


In [212]:
sample = data.sample()
sample_pred = sample.pop('cardio')
sample

,age,gender,BMI,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
53910,3,1,1,1,1,2,2,0,0,1


In [213]:
import time 
start = time.time()
dt.predict(sample, tree)
end = time.time()
end - start

0.0

## SK learn test

In [214]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score

x = data.copy()
y = x.pop('cardio')
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
model = DecisionTreeClassifier(criterion="entropy")
model.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy')

In [215]:
accuracy = metrics.accuracy_score(y_test,model.predict(X_test))
print("Sklearn accuarcy is:", accuracy)

Sklearn accuarcy is: 0.6979285714285715


In [216]:
start = time.time()
model.predict(sample)
end = time.time()
end - start

0.005989789962768555

### Conclusion
My model is less accurate than sklearnaccuracy as my model's accuracy is `65.657142857%` and the sklearn accuracy is `69.23571428571429%`
and my model runs in about `229.13725 secs` which is around `3.817 min` in training but sklearn model runs in `0.1 sec` in training
my model predicts in about `0.01 secs` and sklearn predicts in `0.01689 secs` nearly